# Merge File

In [132]:
# Daftar nama file yang akan digabungkan
file_names = [
    'corpus_Nurul_file_6.txt',      # 267 baris
    'corpus_Hafizha_file_7.txt',    # 225 baris
    'corpus_Aida_file_8.txt',       # 236 baris
    'corpus_Maria_file_9.txt',      # 210 baris
    'corpus_Fadhilah_file_10.txt'   # 230 baris
]

# Nama file output
output_file = 'corpus_kelompok_2.txt'

# Menggabungkan file
with open(output_file, 'w', encoding='utf-8') as outfile:
    for file_name in file_names:
        with open(file_name, 'r', encoding='utf-8') as infile:
            # Membaca konten dari file dan menuliskannya ke file output
            outfile.write(infile.read())
            outfile.write('\n')  # Menambahkan newline setelah setiap file

print(f"File berhasil digabungkan menjadi {output_file}")

File berhasil digabungkan menjadi corpus_kelompok_2.txt


# Pre-processing

In [133]:
import re

# Membaca file teks
input_file = '/content/corpus_kelompok_2.txt'
output_file = 'clean_corpus_kelompok_2.txt'

# Fungsi untuk membersihkan tanda baca yang tidak diinginkan dan menambahkan titik jika belum ada
def clean_and_add_period(line):
    # Lowercasing
    line = line.lower()

    # Menghilangkan tanda baca yang tidak diinginkan
    line = re.sub(r'[,\?!’‘()-]', '', line)  # Hilangkan koma, tanda tanya, tanda seru, karakter yang tidak diinginkan, dan tanda kurung
    line = re.sub(r'\.\.+', '.', line)  # Hilangkan titik ganda menjadi satu titik
    line = line.strip() # Hilangkan spasi di awal/akhir

    # Tambahkan titik di akhir kalimat jika tidak ada
    if not line.endswith('.'):
        return line + '.'
    return line

# Baca file, modifikasi, dan tulis kembali ke file baru
with open(input_file, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Proses setiap baris untuk membersihkan dan menambahkan titik jika belum ada
cleaned_lines = [clean_and_add_period(line) for line in lines]

# Menyimpan hasil ke file baru
with open(output_file, 'w', encoding='utf-8') as file:
    file.write('\n'.join(cleaned_lines))

print("Proses selesai. Hasil perubahan disimpan di 'hasil_perubahan_titik_bersih.txt'.")


Proses selesai. Hasil perubahan disimpan di 'hasil_perubahan_titik_bersih.txt'.


# Tokenisasi

In [134]:
import pandas as pd
import gensim
from gensim.models import Word2Vec

# Convert the cleaned data into a pandas DataFrame
df = pd.DataFrame(cleaned_lines, columns=['Sentence'])

# Show the first few rows of the DataFrame
df.head()

,Sentence
0,kèe meunyo na umu kuwoe u gampông lheuh buet nyoe.
1,kèe han kutém lagèe nyan.
2,uroe nyoe di kèe hana kujak saho.
3,lôn tuan han jeuet lonteuka uroe nyoe.
4,kon baroe lôn han jeuet lonjak tapi uroe nyoe.


In [162]:
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd

# Tokenization words in each sentence
def tokenize(sentences):
    tokenized_sentences = []

    for sentence in sentences:
        # Tokenize each sentence into words using NLTK
        tokens = word_tokenize(sentence)  # Menggunakan word_tokenize dari NLTK

        # Hapus tanda baca (misalnya, titik) dari token
        tokens = [token for token in tokens if token not in ['.', ',', '!', '?', '’', '‘', '-', '(', ')']]

        # Append the tokens as a cleaned sentence
        tokenized_sentences.append(tokens)

    return tokenized_sentences

# Apply tokenization
tokenized_sentences = tokenize(cleaned_lines)

# Convert tokenized sentences into a DataFrame for better viewing
df = pd.DataFrame({'Tokenized_Sentence': tokenized_sentences})

# Show the first few rows of the DataFrame
df.head()

,Tokenized_Sentence
0,"[kèe, meunyo, na, umu, kuwoe, u, gampông, lheuh, buet, nyoe]"
1,"[kèe, han, kutém, lagèe, nyan]"
2,"[uroe, nyoe, di, kèe, hana, kujak, saho]"
3,"[lôn, tuan, han, jeuet, lonteuka, uroe, nyoe]"
4,"[kon, baroe, lôn, han, jeuet, lonjak, tapi, uroe, nyoe]"


In [163]:
# Save the processed data to CSV (optional)
output_path = 'tokenized_aceh_data.csv'
df.to_csv(output_path, index=False)

print(f"Tokenized data has been saved to {output_path}")

Tokenized data has been saved to tokenized_aceh_data.csv


# Embedding

In [164]:
from gensim.models import Word2Vec

# define the model
model_aceh= Word2Vec(
    window=10,
    min_count=1,
    workers=4,
    epochs=10,
)

# Build the vocabulary from the tokenized data
model_aceh.build_vocab(tokenized_sentences, progress_per=1000)

# Train the model
model_aceh.train(tokenized_sentences, total_examples=model_aceh.corpus_count, epochs=model_aceh.epochs)

# Save the model to a file (adjust the path as needed)
model_aceh_path = 'word2vec_aceh.model'
model_aceh.save(model_aceh_path)

print(f"Word2Vec model has been saved to {model_aceh_path}")

# Load the model back (if needed)
model_aceh = Word2Vec.load(model_aceh_path)

Word2Vec model has been saved to word2vec_aceh.model


In [165]:
# Contoh penggunaan model
word = "aneuk"  # Ganti dengan kata yang ingin dicari
try:
    vector = model_aceh.wv[word]
    print(f"Vektor untuk '{word}': {vector}")
except KeyError:
    print(f"Kata '{word}' tidak ditemukan dalam model.")

Vektor untuk 'aneuk': [-0.10518654  0.40187466  0.05241638 -0.07201001  0.12389908 -0.6553
  0.22061779  1.0153809  -0.3403565  -0.442001   -0.20171462 -0.78833145
 -0.22925018  0.33748117  0.29956016 -0.40488577  0.13420711 -0.43084663
 -0.00395685 -0.9190707   0.2666596   0.11902066  0.2535381  -0.19442567
  0.00132269 -0.0299121  -0.21608579 -0.16245532 -0.36138242  0.08861928
  0.5125608   0.06033336  0.02307735 -0.50387603 -0.10326801  0.4550482
  0.1665454  -0.2520448  -0.29140988 -0.65744597 -0.07594479 -0.59401244
 -0.25960755  0.07072486  0.47342643  0.07680864 -0.34274724 -0.14296755
  0.25954118  0.4248429   0.23567776 -0.3873099  -0.0427228  -0.14780676
 -0.2889829   0.11695939  0.24160849 -0.13527597 -0.4435433  -0.01857341
  0.04175976  0.0527241  -0.05627399  0.01276249 -0.33785236  0.505621
  0.09341558  0.4785015  -0.4609872   0.48854244 -0.13063148  0.3167134
  0.38153952 -0.0909666   0.466179    0.16761738  0.11983747 -0.00602063
 -0.28439575  0.17653361 -0.20240098 

In [167]:
# Contoh penggunaan model
word = 'lôn'  # Ganti dengan kata yang diinginkan

try:
    # Mencari kata-kata yang mirip
    similar_words = model_aceh.wv.most_similar(word, topn=10)  # Mencari 10 kata teratas yang mirip

    # Menampilkan hasil
    print(f"Kata yang mirip dengan '{word}':")
    for similar_word, similarity in similar_words:
        print(f"{similar_word}: {similarity:.4f}")
except KeyError:
    print(f"Kata '{word}' tidak ditemukan dalam model.")

Kata yang mirip dengan 'lôn':
bak: 0.9997
nyan: 0.9997
ngon: 0.9997
ureueng: 0.9997
keu: 0.9997
di: 0.9996
lon: 0.9996
nyang: 0.9996
lam: 0.9996
u: 0.9996
